In [ ]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

pd.set_option('display.max_columns', 30)

X = pd.concat([X_train, X_test], axis = 0)
Y = y_train[['gender']] 

X_category = X[['cust_id', '주구매상품', '주구매지점']]
X_serial = X[['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수',
       '주말방문비율', '구매주기']]


# 카테고리는 -> 원핫
X_category = pd.get_dummies(X_category)

# print(X_serial.corr())

import numpy as np
print(X_serial.shape)
print(X_serial.describe())

총구매액1분위수 = X_serial['총구매액'].quantile(0.25)
총구매액3분위수 = X_serial['총구매액'].quantile(0.75)
총구매액사분위편차 = 총구매액3분위수 - 총구매액1분위수
print(총구매액사분위편차)

# 사분위 편차 제거
X_serial = X_serial[
	(X_serial['총구매액'] < (총구매액3분위수 + 총구매액사분위편차*1.5)) &
	(X_serial['총구매액'] > (총구매액1분위수 - 총구매액사분위편차*1.5)) &
	(X_serial['총구매액'] > 0)
]
X_serial['최대구매액'] = np.sqrt(X_serial['최대구매액'])
X_serial['환불금액'] = np.sqrt(X_serial['환불금액'])
X_serial['구매주기'] = np.sqrt(X_serial['구매주기'])
print(X_serial.shape)
print(X_serial.describe())


# X_serial = 이상값, 결측값
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_serial)
X_serial = scaler.transform(X_serial)
X_serial = pd.DataFrame(data = X_serial, columns=['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기'])

X_category = X_category.reset_index()
X_serial = X_serial.reset_index()

X = pd.concat([X_category, X_serial], axis = 1)
X = X.fillna(0)

# index location
X_train = X.iloc[:3500, :] # 까지
X_test  = X.iloc[3500:, :] # 부터

print(X_train.value_counts)

# 알고리즘 적용
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, Y)
pred_train = model.predict(X_train)

pred_test = model.predict(X_test)
# pd.DataFrame(pred_test).to_csv('003000000.csv', index=False)
print(model.score(X_train, y_train[['gender']]))

pred = pd.DataFrame(pred_test, columns = ['pred'])
pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)
